In [1]:
# LLM FROM SCRATCH

In [2]:
# Building a Byte Pair Encoding (BPE) Tokenizer from scratch

In [3]:
# Step 1: Prepare training data
# Note: here we are NOT training a neural network.
# We are just creating a list of tokens from text.
#
# What we need: a text corpus.
# The tokenizer will learn "merge rules" based on how often pairs of characters appear.
#
# Example:
#   "i"   -> token 1
#   "s"   -> token 2
#   "is"  -> token 3
#
# How it works:
# 1. Start with text split into individual characters.
#    (every character is always in the vocabulary)
# 2. Count how often character pairs occur.
# 3. Merge the most frequent pairs into new tokens (subwords).
# 4. Over time, the tokenizer builds a vocabulary that mixes
#    single characters + useful subword tokens.
#
# Important:
# - Characters always remain as fallback tokens.
# - Subwords get priority when tokenizing, making it more efficient.
#
# Concretely:
# - We scan the text and count how many times each pair of characters appears.
# - For example, if the pair "is" appears very often, we create a new token for it.
# - This reduces computation: instead of processing "i" and "s" separately,
#   we can treat "is" as a single token.
# - Note: "i" and "s" still remain in the vocabulary as individual tokens,
#   but whenever the pair "is" exists, it takes priority over the single characters.
#
# Iterative merges:
# - The process continues on top of previously created tokens.
# - For example, if "is" was already merged into a token, and we notice "his"
#   appears frequently, we merge "h" + "is" → "his".
# - Next, if "this" is common, we merge "t" + "his" → "this".
# - This way, the vocabulary gradually grows from characters → subwords → whole words,
#   depending on frequency in the training text.

In [4]:
# Our corpus of data
corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
]

In [5]:
print("Corpus: ")
for doc in corpus:
    print(doc)

Corpus: 
This is the first document.
This document is the second document.
And this is the third one.
Is this the first document?


In [6]:
# Step 2: Initialize vocabulary with unique characters
#
# The first version of our vocabulary is simply all the unique characters
# that appear in the training corpus.
# Each character will be treated as an initial token.
#
# In addition, we add a special end-of-word marker (</w>).
# This marker helps the tokenizer know where words end, so that
# frequent whole words or subwords can be merged properly later.
#
# Example:
#   "this"  →  ["t", "h", "i", "s</w>"]
#   "is"    →  ["i", "s</w>"]
#
unique_chars = set()
for doc in corpus:
    for char in doc:
        unique_chars.add(char)

# Convert to a sorted list so the vocabulary is consistent and reproducible
vocab = list(unique_chars)
vocab.sort()

# Add the special end-of-word token
end_of_word = "</w>"
vocab.append(end_of_word)

In [7]:
print("Initial Vocabulary:")
print(vocab)
print(f"Vocabulary Size: {len(vocab)}")

Initial Vocabulary:
[' ', '.', '?', 'A', 'I', 'T', 'c', 'd', 'e', 'f', 'h', 'i', 'm', 'n', 'o', 'r', 's', 't', 'u', '</w>']
Vocabulary Size: 20


In [8]:
# Step 3: Pre-tokenize the corpus
#
# Goal:
# - Split text into words (by spaces, for simplicity).
# - Break each word into its characters.
# - Add the special end-of-word token (</w>) at the end of every word.
#
# Why?
# - This gives us the initial representation of words as sequences of characters.
# - Example: "This" → ("T", "h", "i", "s", "</w>")
#
# Implementation details:
# - We store each word as a tuple of characters (immutable).
#   Tuples can be used as dictionary keys, unlike lists.
# - We count how many times each word (as a sequence of characters) appears
#   in the whole corpus.
#
# Note:
# - Adding the </w> token ensures that subwords are learned within word
#   boundaries. For example:
#   "document" → ("d", "o", "c", "u", "m", "e", "n", "t", "</w>")
#   This way, if "doc" becomes a frequent subword, it is clear that it
#   belongs inside the word "document" and not across words.
#
word_splits = {}
for doc in corpus:
    words = doc.split(' ')
    for word in words:
        if word:
            # Represent word as characters + </w>
            char_list = list(word) + [end_of_word]
            word_tuple = tuple(char_list)
            
            # Count frequency of this word form
            if word_tuple not in word_splits:
                word_splits[word_tuple] = 0
            word_splits[word_tuple] += 1

print("\nPre-tokenized Word Frequencies:")
print(word_splits)


Pre-tokenized Word Frequencies:
{('T', 'h', 'i', 's', '</w>'): 2, ('i', 's', '</w>'): 3, ('t', 'h', 'e', '</w>'): 4, ('f', 'i', 'r', 's', 't', '</w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '.', '</w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '</w>'): 1, ('s', 'e', 'c', 'o', 'n', 'd', '</w>'): 1, ('A', 'n', 'd', '</w>'): 1, ('t', 'h', 'i', 's', '</w>'): 2, ('t', 'h', 'i', 'r', 'd', '</w>'): 1, ('o', 'n', 'e', '.', '</w>'): 1, ('I', 's', '</w>'): 1, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '?', '</w>'): 1}


In [14]:
# Step 4: Count symbol pair frequencies
#
# Goal:
# - Take the dictionary of word splits we created (word_splits).
# - For each word, look at all adjacent pairs of symbols.
# - Count how many times each pair appears across the entire corpus.
#
# Example:
#   Input: {("T", "h", "i", "s", "</w>"): 2}
#   Output: {("T", "h"): 2, ("h", "i"): 2, ("i", "s"): 2, ("s", "</w>"): 2}

import collections 

def get_pair_stats(splits):
    """
    Count the frequency of adjacent symbol pairs in the word_splits dictionary.
    Example:
        {('T', 'h', 'i', 's', '</w>'): 2, ('i', 's', '</w>'): 3, ...}
    Output:
        {('T', 'h'): 2, ('h', 'i'): 2, ('i', 's'): 5, ...}
    """

    pair_counts = collections.defaultdict(int)
    # there is a difference between the normal dict of python and collections.defaultdict 
    # the difference is: in a normal dict if we try to call a key that doesn't exist,
    # it will throw an error. 
    # but this dictionary will create that key automatically,
    # and the value it assigns to that key will be whatever we pass to the function "int",
    # in this case it assigns zero.

    for word_tuple, freq in splits.items():
        # Example: ('T', 'h', 'i', 's', '</w>'): 2
        # word_tuple = ('T', 'h', 'i', 's', '</w>')
        # freq = 2 
        symbols = list(word_tuple)
        # symbols = ['T', 'h', 'i', 's', '</w>']
        for i in range(len(symbols) - 1):
            # len(symbols) - 1 ensures the last pair doesn't go out of range 
            # because we are accessing symbols[i+1]
            pair = (symbols[i], symbols[i+1])
            # Example: pair = ('T', 'h')
            pair_counts[pair] += freq  # adding the frequency of the pair
            # Example: pair_counts = {('T', 'h'): freq}
    return pair_counts


In [10]:
# Step 5: Merge the most frequent pair
#
# Goal:
# - Take the most frequent pair of symbols (from pair_counts).
# - Go through every word in word_splits.
# - Replace occurrences of that pair with a new merged token.
# - Keep track of merges (so we could undo or review them later if needed).
#
# Example:
#   Input pair_to_merge: ('i', 's')
#   Input word_splits: {
#       ('T', 'h', 'i', 's', '</w>'): 2,
#       ('i', 's', '</w>'): 3,
#       ...
#   }
#   Output new_splits: {
#       ('T', 'h', 'is', '</w>'): 2,
#       ('is', '</w>'): 3,
#       ...
#   }
#
def merge_pair(pair_to_merge, splits):
    """Merges the specified pair in the word splits."""
    new_splits = {} # empty dictionary 
    (first, second) = pair_to_merge # ('i','s') → first = 'i', second = 's'
    merged_token = first + second   # merged_token = 'is' (a string)

    for word_tuple, freq in splits.items():
        # Example: word_tuple = ('T', 'h', 'i', 's', '</w>'), freq = 2
        symbols = list(word_tuple)
        # symbols = ['T', 'h', 'i', 's', '</w>']
        new_symbols = []
        i = 0
        while i < len(symbols):
            if i < len(symbols) - 1 and symbols[i] == first and symbols[i+1] == second:
                new_symbols.append(merged_token)
                # if symbols[i] and symbols[i+1] match the pair_to_merge,
                # replace them with the merged token
                i += 2 
            else:
                # otherwise, just add the current character
                new_symbols.append(symbols[i])
                i += 1
        # store the updated sequence of symbols as a tuple, with the same frequency
        new_splits[tuple(new_symbols)] = freq
    return new_splits

In [11]:
# Step 6: Quick recap of our data structures (with examples from our corpus)
#
# Corpus (toy example):
# [
#   "This is the first document.",
#   "This document is the second document.",
#   "And this is the third one.",
#   "Is this the first document?",
# ]
#
# 1) vocab
#    - Definition: list of ALL unique characters found in the corpus, plus the end-of-word marker </w>.
#    - Example (one possible order, after sorting and then appending </w>):
#      [' ', '.', '?', 'A', 'I', 'T', 'c', 'd', 'e', 'f', 'h', 'i', 'm', 'n', 'o', 'r', 's', 't', 'u', '</w>']
#    - Purpose: this is our initial token set (characters).
#
# 2) word_splits
#    - Definition: a dictionary mapping each WORD (as a tuple of characters + '</w>') to its frequency in the corpus.
#      e.g., { ('T','h','i','s','</w>'): 2, ('i','s','</w>'): 3, ... }
#    - How it’s built: split each line by spaces → take each word → turn into list(word) + ['</w>'] → tuple(...) as key.
#    - Notes on punctuation and case:
#        * Punctuation stays inside the word (e.g., "document." becomes ('d','o','c','u','m','e','n','t','.', '</w>')).
#        * Case is preserved (e.g., "This" vs "this" are different keys).
#    - Concrete examples from our corpus:
#        * ('T','h','i','s','</w>') appears 2 times  → "This" (line 1 and 2)
#        * ('t','h','i','s','</w>') appears 2 times  → "this" (line 3 and 4)
#        * ('i','s','</w>') appears 3 times         → "is"   (lines 1, 2, 3)
#
# 3) get_pair_state(splits)
#    - Input: the word_splits dictionary.
#    - Output: a dictionary that counts ADJACENT symbol pairs INSIDE each word (never across word boundaries).
#      Example output shape: { ('T','h'): 2, ('h','i'): 4, ('i','s'): 7, ('s','</w>'): 7, ... }
#    - How it works:
#        * For each word tuple like ('T','h','i','s','</w>') with freq=2,
#          it adds counts for ('T','h'), ('h','i'), ('i','s'), ('s','</w>') each +2.
#        * It never considers pairs across words (e.g., it does NOT pair the last char of one word with the first of the next).
#    - Concrete intuition with our corpus:
#        * ('i','s') appears inside:
#            - "This" (upper T) → 2 times total
#            - "this" (lower t) → 2 times total
#            - "is" as a word   → 3 times total (pairs: ('i','s') and ('s','</w>'))
#          So ('i','s') can easily sum to 2 + 2 + 3 = 7 in the pair counts.
#
# 4) merge_pair(pair_to_merge, splits)
#    - Input:
#        * pair_to_merge: a tuple like ('i','s') — typically the MOST frequent pair from get_pair_state.
#        * splits: the current word_splits dictionary.
#    - Process:
#        * For every word tuple, find every occurrence of the adjacent pair ('i','s') and replace it with the merged token "is".
#        * Return a NEW dictionary (same shape as word_splits) but with updated word tuples that include the merged token.
#    - Output shape (same as word_splits):
#        * Before:
#            {
#              ('T','h','i','s','</w>'): 2,
#              ('i','s','</w>'): 3,
#              ...
#            }
#        * After merging ('i','s') → "is":
#            {
#              ('T','h','is','</w>'): 2,
#              ('is','</w>'): 3,
#              ...
#            }

In [ ]:
# Step 7: iterative BPE Merging loop

num_merges = 15
# Stores merge rules, e.g., {('a', 'b'): 'ab'} {('T', 'h'): 'Th'}
merges = {}
current_splits = word_splits.copy() # Start with initial word splits {('T', 'h', 'i', 's', '</w>'): 2, ('i', 's', '</w>'): 3, ('t', 'h', 'e', '</w>'): 4, ('f', 'i', 'r', 

print("\n--- Starting BPE Merges ---")
print(f"Initial Splits: {current_splits}")
print("-" * 30)

for i in range(num_merges):
    print(f"\nMerge Iteration {i+1}/{num_merges}")
    # 1. Calculate Pair Frequencies
    pair_stats = get_pair_stats(current_splits) # pair_stats  {('T', 'h'): 2, ('h', 'i'): 5, ('i', 's'): 7, ('s', '</w>'): 8, ('t', 'h'): 7, ('h', 'e'): 4, ('e', '</w>'): 4, ('f', 'i'): 2,
    


--- Starting BPE Merges ---
Initial Splits: {('T', 'h', 'i', 's', '</w>'): 2, ('i', 's', '</w>'): 3, ('t', 'h', 'e', '</w>'): 4, ('f', 'i', 'r', 's', 't', '</w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '.', '</w>'): 2, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '</w>'): 1, ('s', 'e', 'c', 'o', 'n', 'd', '</w>'): 1, ('A', 'n', 'd', '</w>'): 1, ('t', 'h', 'i', 's', '</w>'): 2, ('t', 'h', 'i', 'r', 'd', '</w>'): 1, ('o', 'n', 'e', '.', '</w>'): 1, ('I', 's', '</w>'): 1, ('d', 'o', 'c', 'u', 'm', 'e', 'n', 't', '?', '</w>'): 1}
------------------------------

Merge Iteration 1/15

Merge Iteration 2/15

Merge Iteration 3/15

Merge Iteration 4/15

Merge Iteration 5/15

Merge Iteration 6/15

Merge Iteration 7/15

Merge Iteration 8/15

Merge Iteration 9/15

Merge Iteration 10/15

Merge Iteration 11/15

Merge Iteration 12/15

Merge Iteration 13/15

Merge Iteration 14/15

Merge Iteration 15/15
